# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [7]:
#Your code here
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_csv('homepage_actions.csv')

user_actions = (
    df.groupby(["id", "group"])["action"]
      .agg(lambda s: set(s))
      .reset_index(name="actions")
)
# 1) How many viewers also clicked?
viewed = user_actions["actions"].apply(lambda s: "view" in s)
clicked = user_actions["actions"].apply(lambda s: "click" in s)
viewers_also_clicked = int((viewed & clicked).sum())
print({"viewers_also_clicked": viewers_also_clicked})

# 2) Anomalies: did anyone click who didn't view?
clicked_without_view = int((clicked & ~viewed).sum())
print({"clicked_without_view": clicked_without_view})

# 3) Overlap between control and experiment groups (same id in >1 group)
overlap_mask = df.groupby("id")["group"].nunique() > 1
overlap_ids = overlap_mask.index[overlap_mask]
overlap_user_count = int(len(overlap_ids))
print({"overlap_user_count": overlap_user_count})


{'viewers_also_clicked': 1860}
{'clicked_without_view': 0}
{'overlap_user_count': 0}


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
from math import sqrt, erfc

try:
    clean_user
except NameError:
    # Build user_actions if missing
    try:
        user_actions
    except NameError:
        user_actions = (
            df.groupby(["id", "group"])["action"]
              .agg(lambda s: set(s))
              .reset_index(name="actions")
        )
    # Exclude users appearing in both groups
    overlap_mask = df.groupby("id")["group"].nunique() > 1
    overlap_ids = overlap_mask.index[overlap_mask]
    clean_user = user_actions[~user_actions["id"].isin(overlap_ids)].copy()
    # Restrict to users who viewed
    clean_user = clean_user[clean_user["actions"].apply(lambda s: "view" in s)].copy()

# --- Statistical test: H0 p_exp = p_ctrl vs H1 p_exp > p_ctrl ---
clean_user["clicked_binary"] = clean_user["actions"].apply(lambda s: "click" in s).astype(int)

ctrl = clean_user[clean_user["group"] == "control"]["clicked_binary"]
exp  = clean_user[clean_user["group"] == "experiment"]["clicked_binary"]

x1, n1 = int(exp.sum()), int(exp.count())        # experiment
x2, n2 = int(ctrl.sum()), int(ctrl.count())      # control
p1, p2 = (x1 / n1), (x2 / n2)

p_pool = (x1 + x2) / (n1 + n2)
se = sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
z_stat = (p1 - p2) / se

# One-sided p-value (experiment > control): p = 0.5 * erfc(z / sqrt(2))
pval_one_sided = 0.5 * erfc(z_stat / sqrt(2))

print({
    "exp_clicks": x1, "exp_viewers": n1, "exp_ctr": round(p1, 6),
    "ctrl_clicks": x2, "ctrl_viewers": n2, "ctrl_ctr": round(p2, 6),
    "z_stat": round(z_stat, 4),
    "pval_one_sided": float(pval_one_sided)
})

{'exp_clicks': 928, 'exp_viewers': 2996, 'exp_ctr': 0.309746, 'ctrl_clicks': 932, 'ctrl_viewers': 3332, 'ctrl_ctr': 0.279712, 'z_stat': 2.6186, 'pval_one_sided': 0.004415037788297906}


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [ ]:
#Your code here
# Step 1: Expected clicks in the experiment group under H0 (same CTR as control)

# Subsets among viewers (already filtered in clean_user)
ctrl_view = clean_user[clean_user["group"] == "control"]
exp_view  = clean_user[clean_user["group"] == "experiment"]

# Control CTR (proportion clicked among viewers)
p_ctrl = ctrl_view["actions"].apply(lambda s: "click" in s).astype(int).mean()

# Expected clicks in experiment if it had control CTR
n_exp = len(exp_view)
expected_clicks_exp = n_exp * p_ctrl

print({
    "control_ctr": float(p_ctrl),
    "exp_viewers_n": int(n_exp),
    "expected_clicks_exp_under_H0": float(expected_clicks_exp)
})

{'control_ctr': 0.2797118847539016, 'exp_viewers_n': 2996, 'expected_clicks_exp_under_H0': 838.0168067226891}


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here
actual_clicks_exp = exp_view["actions"].apply(lambda s: "click" in s).astype(int).sum()
z_score = (actual_clicks_exp - expected_clicks_exp) / np.sqrt(n_exp * p_ctrl * (1 - p_ctrl))

print({"z_score": float(z_score)})

{'z_score': 3.6625360854823588}


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here
p_value = 1 - stats.norm.cdf(z_score)

print({"p_value_one_sided": float(p_value)})

{'p_value_one_sided': 0.00012486528006949715}


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
Yes — it matches within rounding (z ≈ 2.62; one-sided p ≈ 0.0044).

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.